In [0]:
import numpy as np
import pandas as pd

In [0]:
dataset = pd.read_csv("do", delimiter="\t")

In [0]:
dataset

,content,type
0,Mình cũng làm ở đây 5 năm và đánh giá như sau:...,2
1,"T đã đọc hết các review, đa phần là chế nhạo b...",1
2,"Mình nhận xét về IT-DPS - Lương tầm 6-8 tr, tr...",1
3,"Các bạn fresher (developer) mới ra trường, đan...",0
4,"Mình là cao cao(anh của cao nhất), mình có 1 v...",0


In [0]:
!wget -N https://raw.githubusercontent.com/stopwords/vietnamese-stopwords/master/vietnamese-stopwords.txt

--2020-05-05 09:47:16--  https://raw.githubusercontent.com/stopwords/vietnamese-stopwords/master/vietnamese-stopwords.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20475 (20K) [text/plain]
Saving to: ‘vietnamese-stopwords.txt’

vietnamese-stopword 100%[===================>]  20.00K  --.-KB/s    in 0.007s  

Last-modified header missing -- time-stamps turned off.
2020-05-05 09:47:16 (2.71 MB/s) - ‘vietnamese-stopwords.txt’ saved [20475/20475]



In [0]:
import re

def process_vn_text(dataset):
    for i in range(len(dataset)):
        review = re.sub(
            "[^a-z0-9A-Z_ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚĂĐĨŨƠàáâãèéêìíòóôõùúăđĩũơƯĂẠẢẤẦẨẪẬẮẰẲẴẶẸẺẼỀỀỂưăạảấầẩẫậắằẳẵặẹẻẽềềểỄỆỈỊỌỎỐỒỔỖỘỚỜỞỠỢỤỦỨỪễệỉịọỏốồổỗộớờởỡợụủứừỬỮỰỲỴÝỶỸửữựỳỵỷỹ]",
            ' ', dataset[i]
        )
        review = review.lower()
        review = review.split()
        review = [
                  word 
                  for word in review 
                  if not word in set(
                      open("vietnamese-stopwords.txt", "r")\
                        .read()\
                        .split("\n")
                  )
        ]
        yield " ".join(review)

In [0]:
corpus = list(process_vn_text(dataset['content']))

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
cv.fit(corpus)
X = cv.transform(corpus).toarray()
y = dataset.iloc[:, 1].values

In [0]:
import tensorflow as tf
y = tf.one_hot(y,3)

In [0]:
inputs = tf.keras.Input(shape=(X.shape[-1],))
hidden = tf.keras.layers.Dense(9, activation=tf.nn.relu)(inputs)
outputs = tf.keras.layers.Dense(y.shape[-1], activation=tf.nn.softmax)(hidden)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [0]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.binary_crossentropy,
    metrics=['accuracy']
)

In [0]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 260)]             0         
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 2349      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 30        
Total params: 2,379
Trainable params: 2,379
Non-trainable params: 0
_________________________________________________________________
